In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/davo/ray_results/_aux_train_daostack_2023-08-17_13-20-17...


2023-08-17 13:20:52,799	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.


Result(
  metrics={'loss': 0.21574559345621547, 'val_loss': nan, 'rprec train': 0.4516187012195587, 'rprec test': 0.1866638958454132, 'p@5 train': 0.6115384697914124, 'p@5 test': 0.21153846383094788, 'r@5 train': 0.11345720291137695, 'r@5 test': 0.1258486658334732, 'should_checkpoint': True, 'done': True, 'trial_id': '029d93e2', 'experiment_tag': '3_trial_index=2,batch_size=9,conv_layers=5,embedding_dim=15,l2=0.0001,learning_rate=0.0054,max_epochs=50'},
  path='/home/davo/ray_results/_aux_train_daostack_2023-08-17_13-20-17/_aux_train_daostack_029d93e2_3_trial_index=2,batch_size=9,conv_layers=5,embedding_dim=15,l2=0.0001,learning_rate=0.0054,max_epochs_2023-08-17_13-20-24',
  checkpoint=Checkpoint(local_path=/home/davo/ray_results/_aux_train_daostack_2023-08-17_13-20-17/_aux_train_daostack_029d93e2_3_trial_index=2,batch_size=9,conv_layers=5,embedding_dim=15,l2=0.0001,learning_rate=0.0054,max_epochs_2023-08-17_13-20-24/checkpoint_000049)
)

In [3]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('p@5 test', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

Index(['loss', 'val_loss', 'rprec train', 'rprec test', 'p@5 train',
       'p@5 test', 'r@5 train', 'r@5 test', 'time_this_iter_s',
       'should_checkpoint', 'done', 'training_iteration', 'trial_id',
       'time_total_s', 'config/__trial_index__', 'config/batch_size',
       'config/conv_layers', 'config/embedding_dim', 'config/l2',
       'config/learning_rate', 'config/max_epochs'],
      dtype='object')


,loss,val_loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,should_checkpoint,...,training_iteration,trial_id,time_total_s,config/__trial_index__,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,config/max_epochs
1,0.212934,NaN,0.436508,0.166715,0.569231,0.213462,0.106345,0.112129,0.175800,True,...,46,21e4a506,7.938328,1,512,5,15,0.000054,0.00539,50
3,0.213819,NaN,0.450422,0.171847,0.619231,0.246154,0.116470,0.135342,0.180599,True,...,43,dc009135,7.224508,0,512,5,15,0.000054,0.00539,50
4,0.248211,NaN,0.400130,0.149757,0.571154,0.207692,0.099729,0.112192,0.182015,True,...,34,e876fafc,6.132464,3,512,5,15,0.000054,0.00539,50
0,0.282165,NaN,0.389379,0.178238,0.519231,0.217308,0.089048,0.125730,0.173828,True,...,19,029d93e2,3.380477,2,512,5,15,0.000054,0.00539,50
2,0.315707,NaN,0.363500,0.151721,0.475000,0.205769,0.076788,0.098339,0.182746,True,...,20,303c4985,3.661164,4,512,5,15,0.000054,0.00539,50


In [4]:
# Group by config (except trial index) and do the mean
non_config_columns = ['config/__trial_index__', 'config/max_epochs']
cfg_columns = [c for c in rdf.columns if c.startswith('config/') and c not in non_config_columns]
non_results_cols = ['should_checkpoint', 'done', 'config/__trial_index__'] + non_config_columns
rdfgb = rdf.drop(columns=non_results_cols).groupby(cfg_columns).mean(numeric_only=True).reset_index()
print(len(rdfgb), "configurations were tested")
rdfgb.sort_values('p@5 test', ascending=False).tail(30)

1 configurations were tested


,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,loss,val_loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,training_iteration,time_total_s
0,512,5,15,0.000054,0.00539,0.254567,NaN,0.407988,0.163656,0.550769,0.218077,0.097676,0.116746,0.178997,32.4,5.667388
